In [1]:
import pandas as pd
import ollama
import os
from tqdm import tqdm
import time

In [2]:
len(os.listdir('/Volumes/T7/WKWSCI-dataset/Article-Bias-Prediction/data/jsons'))

37554

In [3]:
articles=[]
for json in os.listdir('/Volumes/T7/WKWSCI-dataset/Article-Bias-Prediction/data/jsons'):
    articles.append(pd.read_json('/Volumes/T7/WKWSCI-dataset/Article-Bias-Prediction/data/jsons/' + json, orient='index').T)

In [4]:
dataset=pd.concat(articles).reset_index()

In [5]:
dataset['bias'].value_counts()

bias
2    13734
0    13005
1    10815
Name: count, dtype: int64

In [6]:
stopped=pd.read_csv('gen_wo_source-1.csv')

In [7]:
stopped.iloc[12014]['gen_wo_source1']

nan

#### Without media information

In [8]:
gen_wo_source1={}
for idx in tqdm(range(12014, dataset.shape[0])):
    response=ollama.chat(
        model='llama3.1',
        messages=[{'role': 'system',
                  'content': 'Given a headline of news article, write a piece of news article based on the it regardless of what headline is.'},
                  {'role':'user',
                   'content': dataset.iloc[idx]['title']}
        ]
    )
    gen_wo_source1[idx]=response['message']['content']
    if (idx%1000==0) and (idx!=0):
        time.sleep(300)

100%|██████████| 25540/25540 [56:51:39<00:00,  8.01s/it]    


In [10]:
dataset['gen_wo_source1']=dataset.index.map(gen_wo_source1)

In [34]:
all.drop(columns=['index']).to_csv('../../data/ai-generated-fake-news/gen_wo_source-1.csv', index=False)

In [35]:
all=pd.read_csv('../../data/ai-generated-fake-news/gen_wo_source-1.csv')

In [ ]:
#Map the gen_wo_source1 to dataset dataframe

#### With media information

In [100]:
gen_w_source1={}
for idx in tqdm(range(0, dataset.shape[0])):
    response=ollama.chat(
        model='llama3.1',
        messages=[{'role': 'system',
                  'content': 'Given a headline of news article published by {}, write a piece of news article.'.format(dataset.iloc[idx]['source'])},
                  {'role':'user',
                   'content': dataset.iloc[idx]['title']}
        ]
    )
    gen_w_source1[idx]=response['message']['content']